In [1]:
import cv2
import os
import numpy as np

diretorio_base = './'

tipos_de_fraturas = os.listdir(os.path.join(diretorio_base, 'dados'))

largura_desejada = 224
altura_desejada = 224

for tipo_de_fratura in tipos_de_fraturas:

    diretorio_fratura = os.path.join(diretorio_base, 'dados', tipo_de_fratura)

    diretorio_saida = os.path.join(diretorio_base, 'dados_preprocessados', tipo_de_fratura)
    os.makedirs(diretorio_saida, exist_ok=True)
    
    imagens = os.listdir(diretorio_fratura)
    
    for imagem_nome in imagens:
        if not imagem_nome.endswith(".gif"):
            imagem_path = os.path.join(diretorio_fratura, imagem_nome)
            try:
                imagem = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)
                if imagem is not None and imagem.shape[0] > 0 and imagem.shape[1] > 0:
                    imagem_redimensionada = cv2.resize(imagem, (largura_desejada, altura_desejada))

                    imagem_normalizada = imagem_redimensionada / 255.0
                    
                    imagem_saida_path = os.path.join(diretorio_saida, imagem_nome)
                    cv2.imwrite(imagem_saida_path, (imagem_normalizada * 255).astype(np.uint8))
                else:
                    print(f"Imagem inválida: {imagem_path}")
            except Exception as e:
                print(f"Erro ao processar imagem {imagem_nome}: {str(e)}")
        else:
            # Pule imagens .gif
            print(f"Pulando imagem .gif: {imagem_nome}")

Pulando imagem .gif: GII-2-137-g001.gif
Pulando imagem .gif: ijfa-1-001-001.gif
Pulando imagem .gif: rg.2018180064.fig1.gif
Pulando imagem .gif: rg.2018180064.fig2.gif
Pulando imagem .gif: rg.2020190085.fig2a.gif
Pulando imagem .gif: rg.2015140156.fig6a.gif
Pulando imagem .gif: rg.2020190154.fig17a.gif
Pulando imagem .gif: radiol.2017162100.fig5a.gif
Pulando imagem .gif: radiol.2017162100.fig5b.gif
Pulando imagem .gif: traum1c.gif
Pulando imagem .gif: CCASE_44-FF1.gif
Pulando imagem .gif: CCASE_44-FF4.gif
Pulando imagem .gif: A214966_1_En_4_Fig3b_HTML.gif
Pulando imagem .gif: buckle-fx-distal-radius.gif
Pulando imagem .gif: greenstick__distal__fibula..gif
Pulando imagem .gif: rg.342135073.fig4.gif
Pulando imagem .gif: 13244_2014_371_Fig1_HTML.gif
Imagem inválida: ./dados\Hairline Fracture\bjr-85-1016-1148-f01.jpeg
Pulando imagem .gif: rg.2018180073.fig10a.gif
Pulando imagem .gif: rg.2018180073.fig4b.gif
Pulando imagem .gif: rg.2018180073.fig4c.gif
Pulando imagem .gif: rg.2018180073.fig

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D

caminho_base = './dados_preprocessados'

batch_size = 32
image_shape = (224, 224, 1)  # Agora estamos usando apenas 1 canal (tons de cinza)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% dos dados para validação
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',  
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10 
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

model.save('modelo_fraturas_tons_de_cinza.h5')


Found 1341 images belonging to 12 classes.
Found 329 images belonging to 12 classes.
Epoch 1/10
42/42 [==============================] - 98s 2s/step - loss: 2.5511 - accuracy: 0.1051 - val_loss: 2.4513 - val_accuracy: 0.1246
Epoch 2/10
42/42 [==============================] - 95s 2s/step - loss: 2.3786 - accuracy: 0.1879 - val_loss: 2.4030 - val_accuracy: 0.2036
Epoch 3/10
42/42 [==============================] - 94s 2s/step - loss: 2.2159 - accuracy: 0.2782 - val_loss: 2.3475 - val_accuracy: 0.1976
Epoch 4/10
42/42 [==============================] - 96s 2s/step - loss: 1.9503 - accuracy: 0.3818 - val_loss: 2.2830 - val_accuracy: 0.2553
Epoch 5/10
42/42 [==============================] - 97s 2s/step - loss: 1.6718 - accuracy: 0.4787 - val_loss: 2.2288 - val_accuracy: 0.3435
Epoch 6/10
42/42 [==============================] - 97s 2s/step - loss: 1.4115 - accuracy: 0.5690 - val_loss: 2.2088 - val_accuracy: 0.3495
Epoch 7/10
42/42 [==============================] - 98s 2s/step - loss: 1.1

C:\Users\Natan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D

caminho_base = './dados_preprocessados_ajustes_manuais'

batch_size = 32
image_shape = (224, 224, 1) 

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale', 
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10 
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

model.save('modelo_fraturas_apos_ajustes_na_base.h5')

Found 1082 images belonging to 12 classes.
Found 264 images belonging to 12 classes.
Epoch 1/10
34/34 [==============================] - 73s 2s/step - loss: 2.5054 - accuracy: 0.1128 - val_loss: 2.4280 - val_accuracy: 0.1402
Epoch 2/10
34/34 [==============================] - 70s 2s/step - loss: 2.3771 - accuracy: 0.1848 - val_loss: 2.3529 - val_accuracy: 0.1742
Epoch 3/10
34/34 [==============================] - 74s 2s/step - loss: 2.2255 - accuracy: 0.2468 - val_loss: 2.2746 - val_accuracy: 0.3144
Epoch 4/10
34/34 [==============================] - 76s 2s/step - loss: 2.0137 - accuracy: 0.3577 - val_loss: 2.1737 - val_accuracy: 0.3598
Epoch 5/10
34/34 [==============================] - 73s 2s/step - loss: 1.7815 - accuracy: 0.4436 - val_loss: 2.1768 - val_accuracy: 0.3636
Epoch 6/10
34/34 [==============================] - 72s 2s/step - loss: 1.5358 - accuracy: 0.5379 - val_loss: 2.0367 - val_accuracy: 0.4205
Epoch 7/10
34/34 [==============================] - 70s 2s/step - loss: 1.3

C:\Users\Natan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import cv2
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

diretorio_base = './'

tipos_de_fraturas = os.listdir(os.path.join(diretorio_base, 'dados'))

largura_desejada = 224
altura_desejada = 224

data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

for tipo_de_fratura in tipos_de_fraturas:
    diretorio_fratura = os.path.join(diretorio_base, 'dados_preprocessados_ajustes_manuais', tipo_de_fratura)
    
    diretorio_saida = os.path.join(diretorio_base, 'dados_preprocessados_aug', tipo_de_fratura)
    os.makedirs(diretorio_saida, exist_ok=True)
    
    imagens = os.listdir(diretorio_fratura)
    
    for imagem_nome in imagens:

        if not imagem_nome.endswith(".gif"):
            imagem_path = os.path.join(diretorio_fratura, imagem_nome)

            try:

                imagem = cv2.imread(imagem_path, cv2.IMREAD_GRAYSCALE)
                if imagem is not None and imagem.shape[0] > 0 and imagem.shape[1] > 0:
                    imagem_redimensionada = cv2.resize(imagem, (largura_desejada, altura_desejada))
                    imagem_redimensionada = np.expand_dims(imagem_redimensionada, axis=-1)  # Adicione a dimensão de canais (1)

                    batch_images = np.array([imagem_redimensionada])

                    augmented_images = []
                    i = 0
                    for batch in data_generator.flow(batch_images, batch_size=1):
                        augmented_image = batch[0]
                        augmented_images.append(augmented_image)
                        i += 1
                        if i >= 5:  
                            break

                    for j, augmented_image in enumerate(augmented_images):
                        imagem_saida_path = os.path.join(diretorio_saida, f"{imagem_nome.split('.')[0]}_{j}.jpg")
                        cv2.imwrite(imagem_saida_path, augmented_image)
                else:
                    print(f"Imagem inválida: {imagem_path}")
            except Exception as e:
                print(f"Erro ao processar imagem {imagem_nome}: {str(e)}")
        else:
            # Pule imagens .gif
            print(f"Pulando imagem .gif: {imagem_nome}")

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D

caminho_base = './dados_preprocessados_aug'

batch_size = 32
image_shape = (224, 224, 1)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale', 
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10  
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

model.save('modelo_fraturas_apos_ajustes_na_base_com_aug.h5')

Found 5360 images belonging to 12 classes.
Found 1340 images belonging to 12 classes.
Epoch 1/10
168/168 [==============================] - 353s 2s/step - loss: 2.4603 - accuracy: 0.1252 - val_loss: 2.4152 - val_accuracy: 0.1358
Epoch 2/10
168/168 [==============================] - 351s 2s/step - loss: 2.3738 - accuracy: 0.1679 - val_loss: 2.3878 - val_accuracy: 0.1552
Epoch 3/10
168/168 [==============================] - 349s 2s/step - loss: 2.2864 - accuracy: 0.2069 - val_loss: 2.3882 - val_accuracy: 0.1739
Epoch 4/10
168/168 [==============================] - 350s 2s/step - loss: 2.1076 - accuracy: 0.2826 - val_loss: 2.4224 - val_accuracy: 0.1828
Epoch 5/10
168/168 [==============================] - 349s 2s/step - loss: 1.8942 - accuracy: 0.3640 - val_loss: 2.4518 - val_accuracy: 0.2037
Epoch 6/10
168/168 [==============================] - 348s 2s/step - loss: 1.6428 - accuracy: 0.4631 - val_loss: 2.5591 - val_accuracy: 0.2060
Epoch 7/10
168/168 [==============================] - 34

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

caminho_base = './dados_preprocessados_aug'

batch_size = 32
image_shape = (224, 224, 3)  

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',  
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation'
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

model = Sequential()
model.add(base_model) 
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10 
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

model.save('modelo_fraturas_transfer_learning.h5')



Found 5360 images belonging to 12 classes.
Found 1340 images belonging to 12 classes.
Epoch 1/10
168/168 [==============================] - 782s 5s/step - loss: 2.4429 - accuracy: 0.1552 - val_loss: 2.3223 - val_accuracy: 0.2246
Epoch 2/10
168/168 [==============================] - 782s 5s/step - loss: 2.2469 - accuracy: 0.2405 - val_loss: 2.2611 - val_accuracy: 0.2716
Epoch 3/10
168/168 [==============================] - 785s 5s/step - loss: 2.1078 - accuracy: 0.2912 - val_loss: 2.1716 - val_accuracy: 0.3172
Epoch 4/10
168/168 [==============================] - 789s 5s/step - loss: 1.9880 - accuracy: 0.3356 - val_loss: 2.1039 - val_accuracy: 0.3388
Epoch 5/10
168/168 [==============================] - 787s 5s/step - loss: 1.8914 - accuracy: 0.3823 - val_loss: 2.0786 - val_accuracy: 0.3478
Epoch 6/10
168/168 [==============================] - 834s 5s/step - loss: 1.7944 - accuracy: 0.4131 - val_loss: 2.0519 - val_accuracy: 0.3500
Epoch 7/10
168/168 [==============================] - 79

C:\Users\Natan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

caminho_base = './dados_preprocessados_ajustes_manuais'

batch_size = 32
image_shape = (224, 224, 3)  

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',  
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation'
)


base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)


model = Sequential()
model.add(base_model) 
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 10  # Defina o número desejado de épocas
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)


model.save('modelo_fraturas_transfer_learning.h5')



Found 1082 images belonging to 12 classes.
Found 264 images belonging to 12 classes.
Epoch 1/10
34/34 [==============================] - 162s 5s/step - loss: 2.5545 - accuracy: 0.1322 - val_loss: 2.3270 - val_accuracy: 0.1439
Epoch 2/10
34/34 [==============================] - 163s 5s/step - loss: 2.2687 - accuracy: 0.2283 - val_loss: 2.2004 - val_accuracy: 0.2727
Epoch 3/10
34/34 [==============================] - 162s 5s/step - loss: 2.0787 - accuracy: 0.3133 - val_loss: 2.1155 - val_accuracy: 0.3712
Epoch 4/10
34/34 [==============================] - 161s 5s/step - loss: 1.9233 - accuracy: 0.3697 - val_loss: 2.0073 - val_accuracy: 0.3788
Epoch 5/10
34/34 [==============================] - 163s 5s/step - loss: 1.7412 - accuracy: 0.4640 - val_loss: 1.9439 - val_accuracy: 0.4167
Epoch 6/10
34/34 [==============================] - 161s 5s/step - loss: 1.6124 - accuracy: 0.5092 - val_loss: 1.8764 - val_accuracy: 0.4280
Epoch 7/10
34/34 [==============================] - 169s 5s/step - lo

C:\Users\Natan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


SyntaxError: invalid syntax. Perhaps you forgot a comma? (428475996.py, line 42)

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


caminho_base = './dados_preprocessados_ajustes_manuais'


batch_size = 32
image_shape = (224, 224, 3)  # Alterado para 1 canal (tons de cinza)


train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',  
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    caminho_base,
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation'
)


base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)


model = Sequential()
model.add(base_model)  
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 20  
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

model.save('modelo_fraturas_transfer_learning.h5')



Found 1082 images belonging to 12 classes.
Found 264 images belonging to 12 classes.
Epoch 1/20
34/34 [==============================] - 164s 5s/step - loss: 2.5548 - accuracy: 0.1312 - val_loss: 2.2815 - val_accuracy: 0.2197
Epoch 2/20
34/34 [==============================] - 162s 5s/step - loss: 2.1878 - accuracy: 0.2542 - val_loss: 2.1210 - val_accuracy: 0.3409
Epoch 3/20
34/34 [==============================] - 165s 5s/step - loss: 1.9741 - accuracy: 0.3632 - val_loss: 2.0094 - val_accuracy: 0.3598
Epoch 4/20
34/34 [==============================] - 164s 5s/step - loss: 1.7859 - accuracy: 0.4214 - val_loss: 1.9255 - val_accuracy: 0.4356
Epoch 5/20
16/34 [=============>................] - ETA: 26:09:29 - loss: 1.6269 - accuracy: 0.4881

KeyboardInterrupt: 